In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [85]:
# reading the csv archive and saivng it into df name
df= pd.read_csv(r"E:\___DATASCIENCE\___Data Science_henry\M5\M5\Proyecto Integrador\1 Call Center\Call_Center_1999_DataSet.csv", sep=";", low_memory=False)

In [379]:
df.tail(2)

,vru.line,call_id,customer_id,priority,type,date,vru_entry,vru_exit,vru_time,q_start,...,q_time,outcome,ser_start,ser_exit,ser_time,server,startdate,ID,ser_start_hours,has_number_after_colon
444446,AA0216,17214,0,0,IN,1999-12-31,18:30:19,18:30:54,35,0:00:00,...,0,HANG,0:00:00,0:00:00,0,NO_SERVER,334,17214AA0216,0,True
444447,AA0216,17215,0,0,IN,1999-12-31,22:23:35,22:23:45,10,0:00:00,...,0,HANG,0:00:00,0:00:00,0,NO_SERVER,334,17215AA0216,0,True


##### 1° Cada llamada telefónica entrante es ruteada a través del VRU. Hay 6 VRUs etiquetados desde  AA01 a AA06. Cada VRU tiene varias líneas etiquetadas de 1 a 16. Hay un total de 65 líneas. Cada llamada es asignada a un número de VRU y a un número de línea.

In [14]:
# se obserba que el dataset tiene una codificación distinta
df['vru.line'].unique()

array(['AA0101', 'AA0102', 'AA0103', 'AA0104', 'AA0105', 'AA0106',
       'AA0107', 'AA0108', 'AA0109', 'AA0110', 'AA0111', 'AA0112',
       'AA0115', 'AA0116', 'AA0201', 'AA0202', 'AA0203', 'AA0204',
       'AA0205', 'AA0206', 'AA0209', 'AA0210', 'AA0213', 'AA0214',
       'AA0215', 'AA0216', 'AA0113', 'AA0207', 'AA0208', 'AA0114'],
      dtype=object)

##### 2° A cada llamada telefónica entrante se le asigna un “call id”. Aunque son diferentes, los identificadores no son necesariamente consecutivos por estar asignado a diferentes VRUs.

In [15]:
len(df["call_id"].unique())

54471

##### 3° Es la identificación del cliente. Es única por cliente; si el ID es cero, es porque el sistema no pudo identificar a la persona que realiza la llamada (por ejemplo para el caso de los prospectos no se identifican).

In [16]:
# we asume that there are 12910 clients, might be more.
len(df["customer_id"].unique())

12910

##### 4° Hay dos tipos de prioridades: (Alta-)prioridad y Regular:
• 0 y 1 indican clientes no identificados o clientes regulares (los detallaremos más adelante)
• 2 indica clientes de Alta Prioridad.
• Los clientes son servidos en orden según el “Tiempo en Cola” ("Time in Queue").
• A los clientes de Alta Prioridad se les asigna un tiempo de espera de 1.5 minutos al comienzo de su llamada (esto les permite avanzar en la posición de la cola de llamadas). También están exentos de pagar un fee mensual, que los clientes regulares deben pagar.
• No se ha informado a los clientes sobre la existencia de prioridades.
• Hasta Agosto de 1996, todos los clientes tenían la misma prioridad: 0. Las prioridades 1 y 2 fueron incorporadas el 1° de Agosto de 1996. Todavía hay clientes con prioridad 0, pero son tratados como si fueran de prioridad 1. (se definió que los clientes con prioridad 0 corresponden a clientes de prioridad 1 que no realizaron Upgrade a Alta Prioridad (prioridad 2).

 
• Debido a un error en el sistema, el cliente I.D. no fue registrado para aquellos que no esperaron en la cola, Por lo tanto, su prioridad es 0.


In [21]:
df["priority"].unique()

array([2, 0, 1], dtype=int64)

In [70]:
# since the data set contains only 1999 call entries, we can confidently replace 0's with 1's 
df['priority'] = df['priority'].replace(0, 1)

##### 5° Hay 6 tipos diferentes de servicio:
+ PS - Actividad Regular
+ PE - Actividad Regular en inglés
+ IN - Actividad / Consulta por internet
+ NE - Actividad por Acciones (stock exchange)
+ NW - Cliente potencial (prospecto) solicitando información
+ TT – clientes que dejan un mensaje pidiendo al banco que le devuelvan su llamado pero que cuando el sistema automático devuelve el llamado, el agente pasó a estado “ocupado”, dejando al cliente en espera en la cola.


In [18]:
df["type"].unique()

array(['PS', 'NW', 'PE', 'TT', 'IN', 'NE', 'AA', ' TT'], dtype=object)

In [380]:
df[df["type"]=="PS"]['type'].count()

302522

##### 6° año-mes-día

In [108]:
# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

In [24]:
# se obserba que es un string
df["date"].head(5)

0    1999-01-01
1    1999-01-01
2    1999-01-01
3    1999-01-01
4    1999-01-01
Name: date, dtype: object

##### 7° Hora en que la llamada telefónica ingresa al call center. Más específicamente, cada cliente que llama debe identificarse primero lo que se realiza proporcionando a la VRU la ID del cliente. Por lo tanto, esta es la hora en que la llamada ingresa a la VRU.

In [26]:
# se obserba que es un string
df["vru_entry"].head(3)

0    0:00:31
1    0:34:12
2    6:55:20
Name: vru_entry, dtype: object

##### 8° Hora de salida de la VRU: 
1.	A la cola
2.	O directamente para recibir el servicio.
3.	O para dejar el Sistema (abandono).


In [29]:
# se obserba que es un string
df["vru_exit"].head()

0    0:00:36
1    0:34:23
2    6:55:26
3    7:41:26
4    8:03:24
Name: vru_exit, dtype: object

In [25]:
df["vru_exit"].max()

'9:59:59'

### 9° vru_time - 1 a 3 dígitos
Tiempo (en segundos) de espera en la VRU (calculada como vru_time= exit_time – entry_time) .


In [35]:
# 4832 segundos son aproximadamante una hora y media (1 hr 30 min)
df["vru_time"].max()

4832

In [19]:
# the average time in vru is: 10,29 seconds 
sum_vru_time = df['vru_time'].sum()
num_rows = len(df)
column_average = sum_vru_time / num_rows
print("Average: "+ str(round(column_average,2))+" seconds")

Average: 10.29 seconds


##### 10° q_start. Hora en la que se une a la cola. (la llamada queda “en espera”). Este valor es 00:00:00, para clientes que llegan a ponerse en la cola (abandonan cuando están en la VRU).

In [7]:
df["q_start"].describe()

count      444448
unique      62831
top       0:00:00
freq       112589
Name: q_start, dtype: object

##### 11° q_exit. Tiempo (en segundos) en salir de la cola: ya sea porque recibe el servicio o por qué abandona el llamado.

In [37]:
df["q_exit"].describe()

count      444448
unique      62893
top       0:00:00
freq       112588
Name: q_exit, dtype: object

### 12° q_time. Tiempo de espera en la cola (calculado por q_time = q_exit – q_start)

In [7]:
# 28693 segundos son proximadamente 7 horas, 
df["q_time"].max()

28693

In [13]:
# the average time in qeue is: 59 seconds 
sum_q_time = df['q_time'].sum()
num_rows = len(df)
column_average = sum_q_time / num_rows
print("Average: "+ str(round(column_average,2))+" seconds")

Average: 59.0 seconds


##### 13° Outcome - 4,5 o 7 digitos
Hay tres posibles salidas por cada llamada:
1. AGENT: se dio servicio
2. HANG: se cortó la llamada y no se dió servicio
3. PHANTOM: una llamada en la que virtualmente se ignora lo que sucedió(afortunadamente son pocas llamadas en esta situación).


In [3]:
df["outcome"].unique()


array(['HANG', 'AGENT', 'PHANTOM'], dtype=object)

In [18]:
hang = df[df["outcome"]=="HANG"]["outcome"].count()
agent = df[df["outcome"]=="AGENT"]["outcome"].count()
phantom = df[df["outcome"]=="PHANTOM"]["outcome"].count()
print ("the amount of times the outcome is a hang: "+ str(hang)+ " times." )
print ("the amount of times the outcome are answered by an agent: "+ str(agent)+ " times." )
print ("the amount of times the outcome is a virtual ghosting: "+ str(phantom)+ " times." )
print( "df's shape = "+ str(hang+agent+phantom))

the amount of times the outcome is a hang: 87706 times.
the amount of times the outcome is a answered by an agent: 353143 times.
the amount of times the outcome is a virtual ghosting: 3599 times.
df's shape = 444448


##### 14° ser_start - 6 digitos
Hora de comienzo del servicio por un agente.


In [6]:
df["ser_start"].max()

'9:59:59'

##### 15° ser_exit - 6 digitos
Hora del servicio del servicio por un agente.


In [7]:
df["ser_exit"].max()

'9:59:59'

In [10]:
x = df["ser_exit"]

#### 16° ser_time - 1 to 3 digitos
Duración del servicio en segundos (calculada como ser_time = ser_exit – ser_start)


In [68]:
df["ser_time"].max()

61437

In [79]:
df_ser_time = df.sort_values("ser_time", ascending=False)
df_ser_time.shape

(444448, 18)

##### 17° Server - text
Nombre del cliente que atendió la llamada. Este campo es NO_SERVER, si el servicio no fue provisto.


In [13]:
df["server"].unique()

array(['NO_SERVER', 'MICHAL', 'BASCH', 'KAZAV', 'VICKY', 'YITZ', 'SHARON',
       'MORIAH', 'ZOHARI', 'AVNI', 'PINHAS', 'GILI', 'TALI', 'YIFAT',
       'LORI', 'TOVA', 'ELI', 'MIKI', 'NAAMA', 'ANAT', 'IDIT', 'DORIT',
       'SHLOMO', 'BENSION', 'GELBER', 'AVIDAN', 'DARMON', 'EXPRESS',
       'DAVID', 'STEREN', 'ROTH', 'AVI', 'NO_SERVERAMA', 'ANO_SERVERT',
       'MEIR', 'Z1GOLDIE', 'Z2LIOR', 'Z2OFER', 'Z2IFAT', 'Z2EYAL',
       'Z2ARIE', 'Z2NIRIT', 'Z2ELINOR', 'Z2OFERZ', 'GOLDIE', 'ZIPI',
       'Z1SPIEGEL', 'Z2SPIEGEL', 'Z2ASAF', 'BRUHI', 'MOTI', 'Z2ASSAF',
       'MOSHE'], dtype=object)

# EDA EDA

In [54]:
calls_per_day = df_clean.groupby([df_clean['date'].dt.day_name()]).size().reset_index(name='number_of_calls').sort_values('number_of_calls')
calls_per_day.sort_values("number_of_calls", ascending=False)

,date,number_of_calls
3,Sunday,86911
5,Tuesday,81193
4,Thursday,79727
1,Monday,79497
6,Wednesday,77836
0,Friday,28548
2,Saturday,10736


In [109]:
df_clean.sort_values("date", ascending =False)

,ID,vru.line,call_id,priority,type,date,vru_time,q_time,outcome,ser_start,ser_exit,ser_time,server
444447,17215AA0216,AA0216,17215,0,IN,1999-12-31,10,0,HANG,0:00:00,0:00:00,0,NO_SERVER
444134,16976AA0113,AA0113,16976,0,NW,1999-12-31,14,0,AGENT,13:44:30,13:47:11,161,ZOHARI
444122,16964AA0113,AA0113,16964,0,PS,1999-12-31,9,0,AGENT,8:24:11,8:25:29,78,NO_SERVER
444123,16965AA0113,AA0113,16965,0,PS,1999-12-31,15,0,AGENT,8:49:57,8:50:48,51,DORIT
444124,16966AA0113,AA0113,16966,0,PS,1999-12-31,10,0,AGENT,9:17:38,9:18:33,55,YITZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,33511AA0205,AA0205,33511,0,PS,1999-01-01,11,0,AGENT,7:53:43,7:56:26,163,YITZ
399,33510AA0205,AA0205,33510,0,NE,1999-01-01,14,0,AGENT,7:31:23,7:33:12,109,ZOHARI
398,33509AA0205,AA0205,33509,0,NW,1999-01-01,23,0,HANG,0:00:00,0:00:00,0,NO_SERVER
397,32054AA0204,AA0204,32054,0,NW,1999-01-01,59,0,HANG,0:00:00,0:00:00,0,NO_SERVER


In [15]:
df[df["ser_time"] == 0]["ser_time"].count()

87860

In [ ]:
df["startdate"].unique()

0           0
1           0
2          54
3         208
4         107
         ... 
444443    553
444444      0
444445      0
444446      0
444447      0
Name: ser_time, Length: 444448, dtype: int64

In [214]:
def count_duplicates(df, column_name):
    duplicate_count = df[column_name].duplicated().sum()
    return duplicate_count

In [215]:
duplicate_count = count_duplicates(df, 'call_id')
duplicate_count

389977

In [270]:
# creating a new df to save tha columns that i will use
df_clean= df[["customer_id", 'vru.line', "call_id", "priority", "type", "date", "vru_time", "q_time", "outcome","ser_start", "ser_exit", "ser_time", "server"]].copy()

In [318]:
# There are some scientific numbers in the customer_id column so this line will replace them for the 404 number
df_clean["customer_id"] = df_clean["customer_id"].apply(lambda x: int(x) if x.isdigit() else 404)

In [374]:
# to make the average time chart of each type of service
df_types_t= df_clean[["type", "vru_time", "q_time", "ser_time"]].copy()
df_types_t_avg  = df_types_t.groupby('type', as_index=False).mean().round(2)

In [375]:
df_types_t_avg

,type,vru_time,q_time,ser_time
0,TT,8.00,0.00,0.00
1,AA,24.80,0.00,0.00
2,IN,13.96,82.08,295.38
3,NE,8.38,73.35,244.33
4,NW,11.87,76.61,70.02
5,PE,7.85,92.17,177.81
6,PS,10.24,53.00,151.17
7,TT,3.00,19.98,103.36


In [279]:
# To calculate the service level
nivel_servicio = round(((df[df["ser_time"] <= 90]["ser_time"].count())/df.shape[0])*100, 2)
nivel_servicio

52.46

In [273]:
# Extract hours using regex
df_clean['ser_start_hours'] = df_clean['ser_start'].str.extract(r'^(\d+):')

# Convert extracted hours to integers
df_clean['ser_start_hours'] = df_clean['ser_start_hours'].astype(int)

In [202]:
df_clean[df_clean["ser_start_hours"]== 10]["date"].count()

31823

In [129]:
df["ser_start_hours"].unique()

array([ 0,  6,  7,  8,  9, 10, 11, 12, 13, 19, 20, 21, 22, 23, 18, 14, 15,
       16, 17,  4,  5,  3,  1,  2])

In [274]:
# since there is 8 hours shifts (asumption) i will create a column that adds 1, 2, 3 for 1 to 7 shift, 8 to 15 shift and 16 to 23 shift
# and 0 for 0 values since in this time frame there is no call attended. this will make easy to filter the dataframe

def map_hour_band(hour):
    if hour == 0:
        return 0
    elif 1 <= hour <= 7:
        return 1
    elif 8 <= hour <= 15:
        return 2
    elif 16 <= hour <= 23:
        return 3

# Apply the function to create the 'hour_band' column
df_clean['hour_band'] = df_clean['ser_start_hours'].apply(map_hour_band)

In [224]:
value_counts_band = df_clean['hour_band'].value_counts()
value_counts_band

2    214976
3    131072
0     87264
1     11136
Name: hour_band, dtype: int64

In [222]:
value_counts_hours = df_clean['ser_start_hours'].value_counts()
value_counts_hours

0     87264
10    31823
11    29072
9     28886
12    27023
13    26369
14    25553
16    23473
15    23237
8     23013
17    19276
18    17264
19    15912
21    15209
20    14742
22    14177
23    11019
7     10748
6       341
5        19
1        11
4         9
2         6
3         2
Name: ser_start_hours, dtype: int64

In [245]:
df_clean = df_clean.drop(columns='ID')

In [249]:
df_clean = df_clean.sort_values(by= 'ser_time')

In [254]:
value_counts_type = df_clean['type'].value_counts().reset_index()
value_counts_type.columns = ['type', 'count']

In [257]:
value_counts_hours = df_clean['ser_start_hours'].value_counts().reset_index()
value_counts_hours.columns = ['ser_start_hours', 'count']

In [258]:
value_counts_hours

,ser_start_hours,count
0,0,87264
1,10,31823
2,11,29072
3,9,28886
4,12,27023
5,13,26369
6,14,25553
7,16,23473
8,15,23237
9,8,23013


In [220]:
df[df["ser_start_hours"] == 0].sort_values("ser_start_hours")

,vru.line,call_id,customer_id,priority,type,date,vru_entry,vru_exit,vru_time,q_start,...,q_time,outcome,ser_start,ser_exit,ser_time,server,startdate,ID,ser_start_hours,has_number_after_colon
0,AA0101,33116,9664491,2,PS,1999-01-01,0:00:31,0:00:36,5,0:00:36,...,153,HANG,0:00:00,0:00:00,0,NO_SERVER,0,33116AA0101,0,True
267727,AA0214,24644,0,0,PS,1999-08-09,20:39:39,20:41:17,98,00:00:00,...,0,HANG,00:00:00,00:00:00,0,NO_SERVER,212,24644AA0214,0,False
267726,AA0214,24643,0,0,PS,1999-08-09,20:37:18,20:38:56,98,00:00:00,...,0,HANG,00:00:00,00:00:00,0,NO_SERVER,212,24643AA0214,0,False
267725,AA0214,24642,0,0,PS,1999-08-09,20:34:56,20:36:35,99,00:00:00,...,0,HANG,00:00:00,00:00:00,0,NO_SERVER,212,24642AA0214,0,False
267724,AA0214,24641,0,0,PS,1999-08-09,20:32:34,20:34:13,99,00:00:00,...,0,HANG,00:00:00,00:00:00,0,NO_SERVER,212,24641AA0214,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137891,AA0215,18137,26058826,2,PS,1999-05-02,15:34:20,15:34:26,6,15:34:26,...,73,HANG,0:00:00,0:00:00,0,NO_SERVER,120,18137AA0215,0,True
137877,AA0215,18123,0,0,NW,1999-05-02,13:33:27,13:33:37,10,13:33:37,...,0,HANG,0:00:00,0:00:00,0,NO_SERVER,120,18123AA0215,0,True
137875,AA0215,18121,0,0,PS,1999-05-02,13:15:36,13:15:36,0,0:00:00,...,0,HANG,0:00:00,0:00:00,0,NO_SERVER,120,18121AA0215,0,True
137916,AA0215,18162,0,0,NW,1999-05-02,19:12:20,19:12:30,10,19:12:30,...,183,HANG,0:00:00,0:00:00,0,NO_SERVER,120,18162AA0215,0,True


In [241]:
df_without_zero_hours = df_clean[df_clean["hour_band"] != 0]

In [243]:
value_counts_band =df_without_zero_hours["ser_start_hours"].value_counts().reset_index()
value_counts_band

,index,ser_start_hours
0,10,31823
1,11,29072
2,9,28886
3,12,27023
4,13,26369
5,14,25553
6,16,23473
7,15,23237
8,8,23013
9,17,19276


In [264]:
df_without_zero_clients = df[df["customer_id"] != '0']

In [266]:
value_count_client = df_without_zero_clients['customer_id'].value_counts().reset_index()


In [268]:
value_count_client.head(15)

,index,customer_id
0,58859752,1996
1,58704610,1731
2,7191646,1502
3,54128681,1483
4,52867744,1474
5,58029430,1352
6,74780917,1255
7,55279970,1166
8,57918849,1119
9,1574243,1111


In [ ]:
# saving the whole clean df into  csv to use it in the streamlit app
df_clean.to_csv("call_2.csv")